| ![EEW logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/eew.jpg?raw=true) | ![EDGI logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/edgi.png?raw=true) |
|---|---|

#### This notebook is licensed under GPL 3.0. Please visit our Github repo for more information: 
#### The notebook was collaboratively authored by the Environmental Data & Governance Initiative (EDGI) following our authorship protocol: https://docs.google.com/document/d/1CtDN5ZZ4Zv70fHiBTmWkDJ9mswEipX6eCYrwicP66Xw/
#### For more information about this project, visit https://www.environmentalenforcementwatch.org/

## How to Run this Notebook
* If you click on a gray **code** cell, a little “play button” arrow appears on the left. If you click the play button, it will run the code in that cell (“**running** a cell”). The button will animate. When the animation stops, the cell has finished running.
![Where to click to run the cell](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/pressplay.JPG?raw=true)
* You may get a warning that the notebook was not authored by Google. We know, we authored them! It’s okay. Click “Run Anyway” to continue. 
![Error Message](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/warning-message.JPG?raw=true)
* Run all of the cells in a Notebook to make a complete report. Please feel free to look at and **learn about each result as you create it**!

---

# Watershed statistics from ECHO

Here we load some helper code to get us going.

In [1]:
!git clone https://github.com/edgi-govdata-archiving/ECHO_modules.git
%run ECHO_modules/DataSet.py    

# Import code libraries
import urllib.parse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import requests
import csv
import datetime
import folium
from folium.plugins import FastMarkerCluster
import ipywidgets as widgets
from pandas.errors import EmptyDataError
!pip install geopandas
import geopandas
from shapely import wkb

Cloning into 'ECHO_modules'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 97 (delta 44), reused 70 (delta 27), pack-reused 0
Unpacking objects: 100% (97/97), done.
     |████████████████████████████████| 972kB 2.7MB/s 
     |████████████████████████████████| 14.7MB 303kB/s 
     |████████████████████████████████| 10.9MB 49.4MB/s 


In [2]:
sql = 'SELECT * FROM "wbdhu8" WHERE "states" LIKE \'%VT%\''
#sql = 'SELECT row_to_json(fc) FROM ( SELECT \'FeatureCollection\' As type, array_to_json(array_agg(f)) As features FROM (SELECT \'Feature\' As type, ST_AsGeoJSON(w.geom)::json As geometry, row_to_json((\'x\')) As properties FROM "wbdhu8" where "states" like \'%WY%\' As w) As f) As fc'
url='http://apps.tlt.stonybrook.edu/echoepa/?query='
data_location=url+urllib.parse.quote_plus(sql) + '&pg'
print(data_location)
df=None
try:
  df = pd.read_csv(data_location,encoding='iso-8859-1')
except:
  print('nope!')
df

http://apps.tlt.stonybrook.edu/echoepa/?query=SELECT+%2A+FROM+%22wbdhu8%22+WHERE+%22states%22+LIKE+%27%25VT%25%27&pg


,gid,tnmid,metasource,sourcedata,sourceorig,sourcefeat,loaddate,areasqkm,areaacres,referenceg,name,states,huc8,globalid,shape_leng,shape_area,geom
0,125,{37EDDC80-DBFB-4E14-8432-DD91E0431018},NaN,NaN,NaN,NaN,2012-06-11,1780.46,439961.17,NaN,Mettawee River,"NY,VT",4300101,{1F3E5DDB-4D5F-487B-8214-8DBD0A45F667},3.625280,0.198094,0106000020E610000001000000010300000001000000AD...
1,126,{3A4E543B-6CCC-4E3F-BB0D-9CD10F421618},NaN,NaN,NaN,NaN,2019-09-26,7257.45,1793353.30,NaN,Lake Champlain,"CN,NY,VT",4300108,{C578AE8C-9F48-4EEC-8ADB-92529D9D0618},13.143747,0.820664,0106000020E6100000010000000103000000010000002A...
2,127,{729F5BD0-7420-4F4A-BF8B-17AA64882197},NaN,NaN,NaN,NaN,2019-09-26,2565.51,633951.78,NaN,Richelieu River,"CN,NY,VT",4300109,{5A041E47-0BD9-4A45-A5E1-0A35A510F954},3.986822,0.294923,0106000020E610000001000000010300000001000000E7...
3,128,{87D36FF7-DB2A-4720-B9E2-A1887530B6A6},NaN,NaN,NaN,NaN,2019-01-22,2443.11,603705.54,NaN,Otter Creek,VT,4300102,{C3F146D6-1DC8-49A2-A89C-95EEC66F2907},4.165689,0.273151,0106000020E61000000100000001030000000100000017...
4,129,{F5BD2904-A604-4C6F-855E-A12C84294B72},NaN,NaN,NaN,NaN,2019-01-22,2753.75,680466.87,NaN,Winooski River,VT,4300103,{C68E5CBC-B7CD-4F1C-B765-354FE1B1753E},4.467807,0.310596,0106000020E610000001000000010300000001000000A4...
5,130,{BD36AB79-B9DF-4902-92DC-DDD68EDFC97D},NaN,NaN,NaN,NaN,2019-01-22,1868.90,461815.86,NaN,Lamoille River,VT,4300105,{54A612E8-0233-4A76-AD8A-516E849632F1},4.118799,0.211873,0106000020E6100000010000000103000000010000001F...
6,131,{12FE0C83-16BE-4763-920C-B2B651F6139F},NaN,NaN,NaN,NaN,2019-09-26,2213.96,547080.26,NaN,Missiquoi River,"CN,VT",4300107,{64A7D662-4CB4-4577-B811-6BD75D877730},4.196303,0.252446,0106000020E61000000100000001030000000100000078...
7,132,{246B6143-46F9-4848-9981-49B58BF4923B},NaN,NaN,NaN,NaN,2019-09-26,7983.06,1972654.30,NaN,St. Francois River,"CN,ME,NH,VT",4300202,{4118CFE1-4E3B-4B38-A615-E2ECFADCA45A},7.926318,0.916752,0106000020E610000001000000010300000001000000B3...
8,272,{F6F2E9B1-7475-4E6F-A595-57842814F48D},NaN,NaN,NaN,NaN,2017-08-16,2636.71,651544.34,NaN,Middle Connecticut,"MA,NH,VT",1080201,{622E456B-853C-4179-8F00-4DF959735EE0},5.403743,0.289405,0106000020E6100000010000000103000000010000001D...
9,293,{599F86BC-CB5B-4ADA-80F7-38D92D47CAE7},NaN,NaN,NaN,NaN,2017-02-15,2403.14,593829.17,NaN,Waits,"NH,VT",1080103,{462CD7EB-27ED-49AC-AB57-48F0085EF50F},3.509740,0.270734,0106000020E61000000100000001030000000100000021...


In [5]:
# Convert the PostGIS WKB geometry to a shapely object
for index, row in df.iterrows():
  df.at[index,'geom'] = wkb.loads(row["geom"], hex=True)
df

,gid,tnmid,metasource,sourcedata,sourceorig,sourcefeat,loaddate,areasqkm,areaacres,referenceg,name,states,huc8,globalid,shape_leng,shape_area,geom
0,125,{37EDDC80-DBFB-4E14-8432-DD91E0431018},NaN,NaN,NaN,NaN,2012-06-11,1780.46,439961.17,NaN,Mettawee River,"NY,VT",4300101,{1F3E5DDB-4D5F-487B-8214-8DBD0A45F667},3.625280,0.198094,(POLYGON ((-73.17960101455924 43.7959714341032...
1,126,{3A4E543B-6CCC-4E3F-BB0D-9CD10F421618},NaN,NaN,NaN,NaN,2019-09-26,7257.45,1793353.30,NaN,Lake Champlain,"CN,NY,VT",4300108,{C578AE8C-9F48-4EEC-8ADB-92529D9D0618},13.143747,0.820664,(POLYGON ((-73.01248285231867 45.2705867880643...
2,127,{729F5BD0-7420-4F4A-BF8B-17AA64882197},NaN,NaN,NaN,NaN,2019-09-26,2565.51,633951.78,NaN,Richelieu River,"CN,NY,VT",4300109,{5A041E47-0BD9-4A45-A5E1-0A35A510F954},3.986822,0.294923,(POLYGON ((-73.1264003448502 45.99290510881821...
3,128,{87D36FF7-DB2A-4720-B9E2-A1887530B6A6},NaN,NaN,NaN,NaN,2019-01-22,2443.11,603705.54,NaN,Otter Creek,VT,4300102,{C3F146D6-1DC8-49A2-A89C-95EEC66F2907},4.165689,0.273151,(POLYGON ((-73.31087260914717 44.2187524896970...
4,129,{F5BD2904-A604-4C6F-855E-A12C84294B72},NaN,NaN,NaN,NaN,2019-01-22,2753.75,680466.87,NaN,Winooski River,VT,4300103,{C68E5CBC-B7CD-4F1C-B765-354FE1B1753E},4.467807,0.310596,(POLYGON ((-73.07006459597926 44.5912177234938...
5,130,{BD36AB79-B9DF-4902-92DC-DDD68EDFC97D},NaN,NaN,NaN,NaN,2019-01-22,1868.90,461815.86,NaN,Lamoille River,VT,4300105,{54A612E8-0233-4A76-AD8A-516E849632F1},4.118799,0.211873,(POLYGON ((-72.56724040196809 44.8097659450296...
6,131,{12FE0C83-16BE-4763-920C-B2B651F6139F},NaN,NaN,NaN,NaN,2019-09-26,2213.96,547080.26,NaN,Missiquoi River,"CN,VT",4300107,{64A7D662-4CB4-4577-B811-6BD75D877730},4.196303,0.252446,(POLYGON ((-72.25224719516535 45.3933598274571...
7,132,{246B6143-46F9-4848-9981-49B58BF4923B},NaN,NaN,NaN,NaN,2019-09-26,7983.06,1972654.30,NaN,St. Francois River,"CN,ME,NH,VT",4300202,{4118CFE1-4E3B-4B38-A615-E2ECFADCA45A},7.926318,0.916752,(POLYGON ((-71.19162181660334 46.1158333367523...
8,272,{F6F2E9B1-7475-4E6F-A595-57842814F48D},NaN,NaN,NaN,NaN,2017-08-16,2636.71,651544.34,NaN,Middle Connecticut,"MA,NH,VT",1080201,{622E456B-853C-4179-8F00-4DF959735EE0},5.403743,0.289405,(POLYGON ((-72.07287157044379 43.2753884755363...
9,293,{599F86BC-CB5B-4ADA-80F7-38D92D47CAE7},NaN,NaN,NaN,NaN,2017-02-15,2403.14,593829.17,NaN,Waits,"NH,VT",1080103,{462CD7EB-27ED-49AC-AB57-48F0085EF50F},3.509740,0.270734,(POLYGON ((-71.75918082718073 44.5067902600832...


In [6]:
# Create a geodataframe
gdf = geopandas.GeoDataFrame(df, crs = "EPSG:4326", geometry="geom")
gdf

,gid,tnmid,metasource,sourcedata,sourceorig,sourcefeat,loaddate,areasqkm,areaacres,referenceg,name,states,huc8,globalid,shape_leng,shape_area,geom
0,125,{37EDDC80-DBFB-4E14-8432-DD91E0431018},NaN,NaN,NaN,NaN,2012-06-11,1780.46,439961.17,NaN,Mettawee River,"NY,VT",4300101,{1F3E5DDB-4D5F-487B-8214-8DBD0A45F667},3.625280,0.198094,"MULTIPOLYGON (((-73.17960 43.79597, -73.17943 ..."
1,126,{3A4E543B-6CCC-4E3F-BB0D-9CD10F421618},NaN,NaN,NaN,NaN,2019-09-26,7257.45,1793353.30,NaN,Lake Champlain,"CN,NY,VT",4300108,{C578AE8C-9F48-4EEC-8ADB-92529D9D0618},13.143747,0.820664,"MULTIPOLYGON (((-73.01248 45.27059, -73.01162 ..."
2,127,{729F5BD0-7420-4F4A-BF8B-17AA64882197},NaN,NaN,NaN,NaN,2019-09-26,2565.51,633951.78,NaN,Richelieu River,"CN,NY,VT",4300109,{5A041E47-0BD9-4A45-A5E1-0A35A510F954},3.986822,0.294923,"MULTIPOLYGON (((-73.12640 45.99291, -73.12382 ..."
3,128,{87D36FF7-DB2A-4720-B9E2-A1887530B6A6},NaN,NaN,NaN,NaN,2019-01-22,2443.11,603705.54,NaN,Otter Creek,VT,4300102,{C3F146D6-1DC8-49A2-A89C-95EEC66F2907},4.165689,0.273151,"MULTIPOLYGON (((-73.31087 44.21875, -73.30956 ..."
4,129,{F5BD2904-A604-4C6F-855E-A12C84294B72},NaN,NaN,NaN,NaN,2019-01-22,2753.75,680466.87,NaN,Winooski River,VT,4300103,{C68E5CBC-B7CD-4F1C-B765-354FE1B1753E},4.467807,0.310596,"MULTIPOLYGON (((-73.07006 44.59122, -73.06981 ..."
5,130,{BD36AB79-B9DF-4902-92DC-DDD68EDFC97D},NaN,NaN,NaN,NaN,2019-01-22,1868.90,461815.86,NaN,Lamoille River,VT,4300105,{54A612E8-0233-4A76-AD8A-516E849632F1},4.118799,0.211873,"MULTIPOLYGON (((-72.56724 44.80977, -72.56669 ..."
6,131,{12FE0C83-16BE-4763-920C-B2B651F6139F},NaN,NaN,NaN,NaN,2019-09-26,2213.96,547080.26,NaN,Missiquoi River,"CN,VT",4300107,{64A7D662-4CB4-4577-B811-6BD75D877730},4.196303,0.252446,"MULTIPOLYGON (((-72.25225 45.39336, -72.25288 ..."
7,132,{246B6143-46F9-4848-9981-49B58BF4923B},NaN,NaN,NaN,NaN,2019-09-26,7983.06,1972654.30,NaN,St. Francois River,"CN,ME,NH,VT",4300202,{4118CFE1-4E3B-4B38-A615-E2ECFADCA45A},7.926318,0.916752,"MULTIPOLYGON (((-71.19162 46.11583, -71.19170 ..."
8,272,{F6F2E9B1-7475-4E6F-A595-57842814F48D},NaN,NaN,NaN,NaN,2017-08-16,2636.71,651544.34,NaN,Middle Connecticut,"MA,NH,VT",1080201,{622E456B-853C-4179-8F00-4DF959735EE0},5.403743,0.289405,"MULTIPOLYGON (((-72.07287 43.27539, -72.07267 ..."
9,293,{599F86BC-CB5B-4ADA-80F7-38D92D47CAE7},NaN,NaN,NaN,NaN,2017-02-15,2403.14,593829.17,NaN,Waits,"NH,VT",1080103,{462CD7EB-27ED-49AC-AB57-48F0085EF50F},3.509740,0.270734,"MULTIPOLYGON (((-71.75918 44.50679, -71.75871 ..."


In [7]:
# Make the map
m = folium.Map()

w = folium.GeoJson(
    gdf,
    name = "Watersheds",
).add_to(m)
folium.GeoJsonTooltip(fields=["name"]).add_to(w)
#folium.Choropleth(test, fill_color='YlOrBr').add_to(m)

bounds = m.get_bounds()
m.fit_bounds(bounds)

m

Output hidden; open in https://colab.research.google.com to view.

In [8]:
gdf.set_index("name", inplace=True)

hucs = gdf.index.unique()

selection = widgets.Dropdown(
    options= hucs,
    description='Watershed:',
    disabled=False,
)
selection

Dropdown(description='Watershed:', options=('Mettawee River', 'Lake Champlain', 'Richelieu River', 'Otter Cree…

In [12]:
huc = gdf.at[selection.value, "huc8"]
huc = float(huc)

# Get everything we will need from ECHO_EXPORTER in a single DB query.
# We can then use the full dataframe to specialize views of it.
full_echo_data = None

sql = 'select * from "ECHO_EXPORTER" where "FAC_DERIVED_HUC" = ' + str(huc) + ''
print(sql)
try:
    # Don't index.
    full_echo_data = get_data( sql )
except EmptyDataError:
    print("\nThere are no EPA facilities for this query.\n")
full_echo_data

select * from "ECHO_EXPORTER" where "FAC_DERIVED_HUC" = 1080201.0


,REGISTRY_ID,FAC_NAME,FAC_STREET,FAC_CITY,FAC_STATE,FAC_ZIP,FAC_COUNTY,FAC_FIPS_CODE,FAC_EPA_REGION,FAC_INDIAN_CNTRY_FLG,FAC_FEDERAL_FLG,FAC_US_MEX_BORDER_FLG,FAC_CHESAPEAKE_BAY_FLG,FAC_NAA_FLAG,FAC_LAT,FAC_LONG,FAC_MAP_ICON,FAC_COLLECTION_METHOD,FAC_REFERENCE_POINT,FAC_ACCURACY_METERS,FAC_DERIVED_TRIBES,FAC_DERIVED_HUC,FAC_DERIVED_WBD,FAC_DERIVED_STCTY_FIPS,FAC_DERIVED_ZIP,FAC_DERIVED_CD113,FAC_DERIVED_CB2010,FAC_PERCENT_MINORITY,FAC_POP_DEN,FAC_MAJOR_FLAG,FAC_ACTIVE_FLAG,FAC_MYRTK_UNIVERSE,FAC_INSPECTION_COUNT,FAC_DATE_LAST_INSPECTION,FAC_DAYS_LAST_INSPECTION,FAC_INFORMAL_COUNT,FAC_DATE_LAST_INFORMAL_ACTION,FAC_FORMAL_ACTION_COUNT,FAC_DATE_LAST_FORMAL_ACTION,FAC_TOTAL_PENALTIES,...,RCRA_INFORMAL_COUNT,RCRA_FORMAL_ACTION_COUNT,RCRA_DATE_LAST_FORMAL_ACTION,RCRA_PENALTIES,RCRA_LAST_PENALTY_DATE,RCRA_LAST_PENALTY_AMT,RCRA_QTRS_WITH_NC,RCRA_COMPLIANCE_STATUS,RCRA_SNC_FLAG,RCRA_3YR_COMPL_QTRS_HISTORY,SDWA_IDS,SDWA_SYSTEM_TYPES,SDWA_INFORMAL_COUNT,SDWA_FORMAL_ACTION_COUNT,SDWA_COMPLIANCE_STATUS,SDWA_SNC_FLAG,TRI_IDS,TRI_RELEASES_TRANSFERS,TRI_ON_SITE_RELEASES,TRI_OFF_SITE_TRANSFERS,TRI_REPORTER_IN_PAST,FEC_CASE_IDS,FEC_NUMBER_OF_CASES,FEC_LAST_CASE_DATE,FEC_TOTAL_PENALTIES,GHG_IDS,GHG_CO2_RELEASES,DFR_URL,FAC_SIC_CODES,FAC_NAICS_CODES,FAC_DATE_LAST_INSPECTION_EPA,FAC_DATE_LAST_INSPECTION_STATE,FAC_DATE_LAST_FORMAL_ACT_EPA,FAC_DATE_LAST_FORMAL_ACT_ST,FAC_DATE_LAST_INFORMAL_ACT_EPA,FAC_DATE_LAST_INFORMAL_ACT_ST,FAC_FEDERAL_AGENCY,TRI_REPORTER,FAC_IMP_WATER_FLG,EJSCREEN_FLAG_US
0,110070385292,CONWAY RD MOBIL,242 CONWAY RD,SOUTH DEERFIELD,MA,1373,FRANKLIN,25011,1,N,NaN,NaN,NaN,Y,42.497370,-72.637050,RCRA-IC-MN-N.png,ADDRESS MATCHING-HOUSE NUMBER,CENTER OF A FACILITY OR STATION,30,NaN,1080201,10802010604,25011,1373,2,250110409002062,5.526,153.40,NaN,Y,NNN,0,NaN,NaN,0,NaN,0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,No Violation Identified,N,____________,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://echo.epa.gov/detailed-facility-report?f...,NaN,447110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1,110003415750,SHERWIN WILLIAMS 5661,2221 NORTHAMPTON ST,HOLYOKE,MA,1040,HAMPDEN,25013,1,N,NaN,NaN,NaN,Y,42.186127,-72.630397,RCRA-IC-MN-N.png,ADDRESS MATCHING-HOUSE NUMBER,ENTRANCE POINT OF A FACILITY OR STATION,50,NaN,1080201,10802010702,25013,1040,1,250138121042013,39.573,2722.15,NaN,Y,NNN,0,NaN,NaN,0,NaN,0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,No Violation Identified,N,____________,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://echo.epa.gov/detailed-facility-report?f...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
2,110043778016,TOMS AUTO SERVICE OF KEENE,122 WATER ST,KEENE,NH,3431,CHESHIRE,33005,1,N,NaN,NaN,NaN,NaN,42.930250,-72.273230,RCRA-IC-MN-N.png,ADDRESS MATCHING-HOUSE NUMBER,CENTER OF A FACILITY OR STATION,30,NaN,1080201,10802010202,33005,3431,2,330059711002020,5.832,790.74,NaN,NaN,NNN,0,NaN,NaN,0,NaN,0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,No Violation Identified,N,____________,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://echo.epa.gov/detailed-facility-report?f...,NaN,811111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
3,110002012599,GERMAN GERENA SCHOOL,200 BIRNIE AVE.,SPRINGFIELD,MA,1107,HAMPDEN,25013,1,N,NaN,NaN,NaN,Y,42.118550,-72.610230,MULT-IC-MN-N.png,ADDRESS MATCHING-HOUSE NUMBER,CENTER OF A FACILITY OR STATION,30,NaN,1080201,10802010702,25013,1107,1,250138006003003,55.321,4355.63,NaN,Y,NNN,0,NaN,NaN,0,02/05/1993,0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,No Violation Identified,N,____________,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://echo.epa.gov/detailed-facility-report?f...,8211,611110,NaN,NaN,NaN,NaN,NaN,02/05/1993,NaN,NaN,NaN,Y
4,110057594857,BARKER STEEL LLC,73 OLD STATE RD,SOUTH DEERFIELD,MA,1373,FRANKLIN,25011,1,N,NaN,NaN,NaN,Y,42.471060,-72.612550,MULT-IC-MN-3.png,ADDRESS MATCHING-HOUSE NUMBER,CENTER OF A FACILITY OR STATION,30,NaN,1080201,10802010604,25011,1373,2,250110409004011,10.035,238.06,NaN,Y,NNN,1,04/11/2018,837.0,1,06